In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24714
2,Huelva,Confirmados PDIA 14 días,245
3,Huelva,Tasa PDIA 14 días,"47,74246351111717"
4,Huelva,Confirmados PDIA 7 días,124
5,Huelva,Total Confirmados,24934
6,Huelva,Curados,17328
7,Huelva,Fallecidos,315


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24714.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6568.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 8990 personas en los últimos 7 días 

Un positivo PCR cada 3196 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24714.0,124.0,245.0,513170.0,24.163533,47.742464,51.0
Condado-Campiña,6931.0,51.0,97.0,156231.0,32.643970,62.087550,25.0
Huelva-Costa,15072.0,72.0,144.0,289548.0,24.866343,49.732687,25.0
Ayamonte,1217.0,23.0,31.0,21104.0,108.984079,146.891585,12.0
Almonte,875.0,11.0,27.0,24507.0,44.885135,110.172604,7.0
Huelva (capital),6568.0,16.0,45.0,143837.0,11.123703,31.285413,5.0
Moguer,834.0,5.0,14.0,21867.0,22.865505,64.023414,4.0
San Juan del Puerto,363.0,5.0,5.0,9411.0,53.129317,53.129317,4.0
Bollullos Par del Condado,617.0,7.0,8.0,14387.0,48.655036,55.605755,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,97.0,10.0,16.0,3092.0,323.415265,517.464424,2.0
Lucena del Puerto,159.0,6.0,12.0,3261.0,183.992640,367.985281,2.0
San Bartolomé de la Torre,200.0,7.0,9.0,3761.0,186.120713,239.298059,1.0
San Silvestre de Guzmán,12.0,1.0,1.0,614.0,162.866450,162.866450,NaN
Ayamonte,1217.0,23.0,31.0,21104.0,108.984079,146.891585,12.0
Almonte,875.0,11.0,27.0,24507.0,44.885135,110.172604,7.0
Cartaya,957.0,9.0,20.0,20083.0,44.814022,99.586715,2.0
Palma del Condado (La),978.0,2.0,9.0,10801.0,18.516804,83.325618,1.0
Moguer,834.0,5.0,14.0,21867.0,22.865505,64.023414,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lepe,1996.0,2.0,10.0,27880.0,7.173601,35.868006,2.0,0.200000
Palma del Condado (La),978.0,2.0,9.0,10801.0,18.516804,83.325618,1.0,0.222222
Aljaraque,925.0,1.0,4.0,21474.0,4.656794,18.627177,0.0,0.250000
Huelva (capital),6568.0,16.0,45.0,143837.0,11.123703,31.285413,5.0,0.355556
Moguer,834.0,5.0,14.0,21867.0,22.865505,64.023414,4.0,0.357143
Almonte,875.0,11.0,27.0,24507.0,44.885135,110.172604,7.0,0.407407
Cartaya,957.0,9.0,20.0,20083.0,44.814022,99.586715,2.0,0.450000
Palos de la Frontera,451.0,2.0,4.0,11742.0,17.032873,34.065747,0.0,0.500000
Huelva-Costa,15072.0,72.0,144.0,289548.0,24.866343,49.732687,25.0,0.500000
